# Age determination 🔎

Сетевой супермаркет «Хлеб-Соль» внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:

- анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
- контролировать добросовестность кассиров при продаже алкоголя.

Необходимо построить модель, которая по фотографии определит приблизительный возраст человека. В нашем распоряжении набор фотографий людей с указанием возраста.

Перед нами задача регрессии. На выходе будет 1 слой с ReLU функцией активации, чтобы, например привести к 0 все отрицательные числа.

Данные взяты с сайта [ChaLearn Looking at People](http://chalearnlap.cvc.uab.es/dataset/26/description/).

**Содержание**<a id='toc0_'></a>    
1. [Подготовка ](#toc1_)    
1.1. [Библиотеки ](#toc1_1_)    
1.2. [Конфигурация ](#toc1_2_)    
2. [Данные ](#toc2_)    
2.1. [Загрузка ](#toc2_1_)    
2.2. [Распределение по возрастам ](#toc2_2_)    
2.3. [Визуальный осмотр ](#toc2_3_)    
2.4. [Выводы по данным ](#toc2_4_)    
3. [Предсказание ](#toc3_)    
3.1. [Загрузчики ](#toc3_1_)    
3.2. [Модель ](#toc3_2_)    
4. [Результаты](#toc4_)    
4.1. [Проверка на тесте](#toc4_1_)    
5. [Итоговый вывод ](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>Подготовка  [&#8593;](#toc0_)

### 1.1. <a id='toc1_1_'></a>Библиотеки  [&#8593;](#toc0_)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from termcolor import colored
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense,
                                     GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam

### 1.2. <a id='toc1_2_'></a>Конфигурация  [&#8593;](#toc0_)

In [ ]:
IMAGE_SIZE = (224, 224)
RANDOM_STATE = 27
BATCH_SIZE = 32
EPOCHS = 10

## 2. <a id='toc2_'></a>Данные  [&#8593;](#toc0_)

### 2.1. <a id='toc2_1_'></a>Загрузка  [&#8593;](#toc0_)

In [ ]:
y_train = pd.read_csv('./data/gt_train.csv', usecols=['file_name', 'real_age']).drop_duplicates()
y_train.head()

In [ ]:
y_train.info()

In [ ]:
y_train.describe()

### 2.2. <a id='toc2_2_'></a>Распределение по возрастам  [&#8593;](#toc0_)

In [ ]:
plt.figure(figsize=(15, 5))
y_train['real_age'].hist(bins=50)
plt.title('Гистограмма распределения возрастов')
plt.xlabel('Возраст, лет')
plt.ylabel('Количество, шт.')
plt.show()

### 2.3. <a id='toc2_3_'></a>Визуальный осмотр  [&#8593;](#toc0_)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=y_train,
        directory='./data/train',
        x_col='file_name',
        y_col='real_age',
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='raw',
        seed=RANDOM_STATE)

In [ ]:
images, labels = next(train_gen_flow)
images = images[:10]
labels = labels[:10]

In [ ]:
rows, cols = 5, 2

In [ ]:
fig, axes = plt.subplots(rows, cols, figsize=(10, 5 * rows))

for i, (img, label) in enumerate(zip(images, labels)):
    row = i // cols
    col = i % cols
    ax = axes[row, col]
    ax.imshow(img)
    ax.set_title(f'Age = {label}')
    ax.axis('off')

plt.tight_layout()
plt.show()

### 2.4. <a id='toc2_4_'></a>Выводы по данным  [&#8593;](#toc0_)

Представлены изображения людей обоих полов и самых разных возрастов. Присутствуют изображения, которые повернуты и/или обрезаны. Распределение возрастов смещено влево (в сторону меньшего возраста). Также заметны пики на юбилейных годах (20, 25, 30...). Возможно, это связано с округлением при сборе информации.

## 3. <a id='toc3_'></a>Предсказание  [&#8593;](#toc0_)

### 3.1. <a id='toc3_1_'></a>Загрузчики  [&#8593;](#toc0_)

In [ ]:
y_train = pd.read_csv('./data/gt_train.csv', usecols=['file_name', 'real_age']).drop_duplicates()

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=20)

train_datagen_flow = train_datagen.flow_from_dataframe(
    dataframe=y_train,
    directory='./data/train',
    x_col='file_name',
    y_col='real_age',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    seed=RANDOM_STATE)

In [ ]:
y_valid = pd.read_csv('./data/gt_valid.csv', usecols=['file_name', 'real_age']).drop_duplicates()

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen_flow = valid_datagen.flow_from_dataframe(
    dataframe=y_valid,
    directory='./data/valid',
    x_col='file_name',
    y_col='real_age',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    seed=RANDOM_STATE)

In [ ]:
y_test = pd.read_csv('./data/gt_test.csv', usecols=['file_name', 'real_age']).drop_duplicates()

test_datagen = ImageDataGenerator(rescale=1./255)

test_datagen_flow = test_datagen.flow_from_dataframe(
    dataframe=y_test,
    directory='./data/test',
    x_col='file_name',
    y_col='real_age',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    seed=RANDOM_STATE)

### 3.2. <a id='toc3_2_'></a>Модель  [&#8593;](#toc0_)

In [ ]:
backbone = ResNet50(weights='imagenet',
                    include_top=False,
                    input_shape=(224, 224, 3))

backbone.trainable = False

model = Sequential([
    backbone,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(1, activation='relu')
])

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
                loss='mse',
                metrics=['mae'])

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(colored('Обнаружено GPU:', 'green'), gpus)
else:
    print(colored('GPU не обнаружено. Будет использован CPU.', 'red'))

In [ ]:
history = model.fit(
    train_datagen_flow,
    steps_per_epoch=len(train_datagen_flow),
    validation_data=valid_datagen_flow,
    validation_steps=len(valid_datagen_flow),
    epochs=EPOCHS,
    verbose=2
)

## 4. <a id='toc4_'></a>Результаты [&#8593;](#toc0_)

### 4.1. <a id='toc4_1_'></a>Проверка на тесте [&#8593;](#toc0_)

In [ ]:
test_loss, test_mae = model.evaluate(test_datagen_flow, steps=len(test_datagen_flow))
print(f"test loss: {test_loss}, test MAE: {test_mae}")

## 5. <a id='toc5_'></a>Итоговый вывод  [&#8593;](#toc0_)

В рамках данного проекта мы работали с датасетом, содержащим изображения людей. Целевой признак - их возраст. В датасете представлены изображения людей обоих полов и самых разных возрастов.

В качестве модели использовался бэкбон *ResNet50* с весами от *imagenet*, один скрытый Dense слой с `256` нейронами и 1 выходной слой с ожидаемо `1` нейроном. Использовалась ReLU функция активации в обоих слоях. В результате, получена метрика `MSE ~ 7`, что лучше, чем требовалось.

Однако использовать такую модель в реальной жизни не стоит. Ошибка в `7` лет довольна критична для целей заказчика. Ведь у людей 7 и 14 лет совершенно разные "интересные им" товары. Модель могла бы быть относительно полезной для людей старшей возрастной категории - люди в 50 и 57 покупают примерно одно.